In [ ]:
!pip install pvlib

## Etapas da Aula
- Funções para Extração de Parâmetros e Resolução da Equação Característica da Curva IV (De Soto)

- Características do Módulo FV e Condição de STC

1) Extração de Parâmetros dos Pontos da Curva IV do Datasheet (Métodos da De Soto)

2) Resolvendo a Equação Característica do SDM 5 P (Métodos do Lambert-W, Newton-Raphson e Bishop)

-> Desafio 1: Adaptar a Equação Característica do SDM 5-Param para:
- 5 módulos em Série
- 3 Módulos em Paralelo

-> Desafio 2: Determinem o Fator de Idealidade do Módulo

-> Desafio 3: Fazer o Método do CEC para Extração de Parâmetros do Datasheet e Cálculo de Parâmetros do SDM

3) Modelagem da Curva IV com a Variação de Temperatura e Irradiância

4) Eficiência do Módulo Variando a Irradiância e Temperatura

-> Desafio 4: Façam a Variação para Temperatura, Mantendo a Irradiância Constante em 1000 W/m²

5) Inserindo Dados de Irradiância e Temperatura Modelados com Dados Medidos em Campo

-> Desafio 5: Façam a Modelagem em Máxima Potência usando o Método de OPC da De Soto

-> Desafio 6: Façam o Gráfico de Eficiência do Módulo com os Dados Simulados de POA e Temp Cell

6) Relação de Irradiância e Potência

7) Eficiência ao Longo de um Dia

In [ ]:
import pvlib
from pvlib import pvsystem
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.constants import Boltzmann, elementary_charge

### Funções para Extração de Parâmetros e Resolução da Equação Característica da Curva IV (De Soto)

In [ ]:
# Inputs (Datasheet ou Curva IV Medida sob STC)


In [ ]:
# Função para Cálculo dos Parâmetros em OPC


In [ ]:
#Constantes
EgRef=1.121 #Valence energy band-gap for Crystalline Silicon
dEgdT=- 0.0002677 #
Tref = 25 #Temperature (°C) under STC
Gref = 1000 #Irradiance (W/m²) under STC
k = Boltzmann
q = elementary_charge

### Características do Módulo FV e Condição de STC

In [ ]:
Vmp = 30.8 #Voltage at Maximum Power
Imp = 8.75 #Current at Maximum Power
Voc = 37.9 #Open-Circuit Voltage
Isc = 9.32 #Short-Circuit Current
alpha = 0.05*Isc/100 # Alpha_isc (%/A) * Short-Circuit Current
beta = (-0.31)*Voc/100 # Beta_voc (%/V) * Open-Circuit Voltage
gamma = -0.41
Ns = 60 #Number of PV Cells in Series
A_c = 1.6368 #The Area of PV Module's Surface

In [ ]:
irrad = 1000
temp_cell = 25

### 1) Extração de Parâmetros dos Pontos da Curva IV do Datasheet (Métodos da De Soto)

### 2) Resolvendo a Equação Característica do SDM 5 P (Métodos do Lambert-W, Newton-Raphson e Bishop)

### Desafio 1: Adaptar a Equação Característica do SDM 5-Param para:

- 5 módulos em Série

- 3 Módulos em Paralelo

### Desafio 2: Determinem o Fator de Idealidade do Módulo

### Desafio 3: Fazer o Método do CEC para Extração de Parâmetros do Datasheet e Cálculo de Parâmetros do SDM

In [ ]:
!pip install NREL-PySAM

### 3) Modelagem da Curva IV com a Variação de Temperatura e Irradiância

### 4)  Eficiência do Módulo Variando a Irradiância e Temperatura

### Desafio 4: Façam a Variação para Temperatura, Mantendo a Irradiância Constante em 1000 W/m²

### 5) Inserindo Dados de Irradiância e Temperatura Modelados com Dados Medidos em Campo

In [ ]:
#Import Data
#Localização da Universidade de Oregon

df_weather = pvlib.iotools.read_midc_raw_data_from_nrel('UOSMRL',               # Station id
                                                    pd.Timestamp('20230601'),   # Start date YYYYMMDD
                                                    pd.Timestamp('20230601'))   # End date  YYYYMMDD

df_weather.head(3)

In [ ]:
df_weather = df_weather[['Global CMP22 [W/m^2]', 'Diffuse [W/m^2]',
                         'Direct CHP1 [W/m^2]','Air Temperature [deg C]', 'Avg Wind Speed @ 10m [m/s]']]
# Rename the columns
df_weather.columns = ['ghi', 'dhi', 'dni', 'temp_air', 'wind_speed']

In [ ]:
latitude = 44.0467
longitude = -123.0743
altitude = 133.8

loc = pvlib.location.Location(latitude = latitude, longitude = longitude)

solpos = loc.get_solarposition(df_weather.index)

GHI = loc.get_clearsky(df_weather.index, model = 'simplified_solis')

#Estimação de Perdar por Sombreamento
plt.figure(figsize=(15,5))
df_weather['ghi'].plot(label = 'Experimental data')
GHI['ghi'].plot(label = 'Clearsky Model')
print((GHI['ghi']-df_weather['ghi']).sum())

In [ ]:
#POA no Plano do Módulo
surface_tilt = 23
surface_azimuth = 180
solar_zenith = solpos['zenith']
solar_azimuth = solpos['azimuth']
dni = df_weather['dni']
ghi = df_weather['ghi']
dhi = df_weather['dhi']
dni_extra = pvlib.irradiance.get_extra_radiation(df_weather.index)
airmass_relative = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])
pressure = pvlib.atmosphere.alt2pres(altitude = altitude)
airmass = pvlib.atmosphere.get_absolute_airmass(airmass_relative, pressure)

POA_Irradiance = pvlib.irradiance.get_total_irradiance(surface_tilt,
                                                  surface_azimuth,
                                                  solar_zenith,
                                                  solar_azimuth,
                                                  dni,
                                                  ghi,
                                                  dhi,
                                                  dni_extra=dni_extra,
                                                  airmass=airmass,
                                                  albedo=0.25,
                                                  surface_type=None,
                                                  model='perez', #'isotropic', 'klucher', 'haydavies', 'reindl', 'king', 'perez', 'perez-driesse'
                                                  model_perez='allsitescomposite1990') #Used only for Perez Model)
plt.figure(figsize=(15,5))
POA_Irradiance['poa_global'].plot()
print(POA_Irradiance['poa_global'].max())

In [ ]:
#Temperatura da Célula FV
PV_Cell_Temp = pvlib.temperature.pvsyst_cell(poa_global = POA_Irradiance['poa_global'],
                                             temp_air = df_weather['temp_air'], wind_speed = df_weather['wind_speed'],
                                             u_c = 29.0, u_v=0.0,
                                             module_efficiency=0.1701,
                                             alpha_absorption=0.9)

plt.figure(figsize = (15,6))
PV_Cell_Temp.plot(label = 'PV Cell Temperature')
df_weather['temp_air'].plot(label = 'Air Temperature')
plt.legend()

### Desafio 5: Façam a Modelagem em Máxima Potência usando o Método de OPC da De Soto

### Desafio 6: Façam o Gráfico de Eficiência do Módulo com os Dados Simulados de POA e Temp Cell

### 6) Relação de Irradiância e Potência

### 7) Eficiência ao Longo de um Dia